## Individual Battery Cycle Analysis

This notebook extracts the core logic for plotting individual cycle data from the `BatteryDataTool.py` application. 

### Instructions:
1.  Fill in the parameters in the **Configuration** cell below.
2.  Run all cells to process the data and generate the graphs.

### 1. Configuration

In [ ]:
# --- User Configuration ---

# Enter the absolute path to the specific raw data folder for a single battery test.
# Example: 'C:/Users/Ryu/Python_project/data/battery251027/Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc'
RAW_DATA_PATH = 'C:/Users/Ryu/Python_project/data/battery251027/Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30'

# Enter the cycle number you want to analyze.
CYCLE_NUMBER = 1

# Enter the rated capacity in mAh. If set to 0, the script will try to infer it from the folder name.
RATED_CAPACITY_MAH = 0

# Set the initial C-rate for capacity calculation if RATED_CAPACITY_MAH is 0.
INITIAL_C_RATE = 0.5

# Voltage cutoff for plotting.
VOLTAGE_CUTOFF = 3.0

# Smoothing degree for dQ/dV calculation. If 0, it's auto-calculated.
SMOOTHING_DEGREE = 0

### 2. Imports and Helper Functions

These are the necessary functions extracted from `BatteryDataTool.py` to process the raw data.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.simplefilter("ignore")
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

def check_cycler(raw_file_path):
    """Checks if the cycler is PNE or Toyo based on the presence of a 'Pattern' folder."""
    # This is a simplified check. The original code's check is more complex.
    # We will determine the cycler by trying to read the data in both formats.
    if os.path.isdir(os.path.join(raw_file_path, 'Restore')):
        return 'PNE'
    # A better check might be to look for specific file names like 'capacity.log' for Toyo
    elif os.path.exists(os.path.join(raw_file_path, '..', 'capacity.log')) or any(f.isdigit() for f in os.listdir(raw_file_path)):
        return 'Toyo'
    return 'Unknown'

def name_capacity(data_file_path):
    """Extracts capacity from a file path string."""
    raw_file_path = re.sub(r'[._@\(\)]', ' ', data_file_path)
    match = re.search(r'(\d+([\-.]\d+)?)mAh', raw_file_path)
    if match:
        min_cap = match.group(1).replace('-', '.')
        return float(min_cap)
    return None

def toyo_read_csv(*args):
    """Reads CSV data for Toyo cyclers."""
    if len(args) == 1:
        filepath = os.path.join(args[0], "capacity.log")
        skiprows = 0
    else:
        filepath = os.path.join(args[0], f"{args[1]:06d}")
        skiprows = 3
    if os.path.isfile(filepath):
        try:
            return pd.read_csv(filepath, sep=",", skiprows=skiprows, engine="c", encoding="cp949", on_bad_lines='skip')
        except Exception as e:
            print(f"Error reading {filepath}: {e}")
            return None
    return None

def toyo_Profile_import(raw_file_path, cycle):
    df = pd.DataFrame()
    dataraw = toyo_read_csv(raw_file_path, cycle)
    if dataraw is not None and not dataraw.empty:
        if "PassTime[Sec]" in dataraw.columns:
            if "Temp1[Deg]" in dataraw.columns:
                df.dataraw = dataraw[["PassTime[Sec]", "Voltage[V]", "Current[mA]", "Condition", "Temp1[Deg]"]]
            else:
                df.dataraw = dataraw[["PassTime[Sec]", "Voltage[V]", "Current[mA]", "Condition", "TotlCycle"]]
                df.dataraw.columns = ["PassTime[Sec]", "Voltage[V]", "Current[mA]", "Condition", "Temp1[Deg]"]
        elif "Passed Time[Sec]" in dataraw.columns:
            df.dataraw = dataraw[["Passed Time[Sec]", "Voltage[V]", "Current[mA]", "Condition", "Temp1[deg]"]]
            df.dataraw.columns = ["PassTime[Sec]", "Voltage[V]", "Current[mA]", "Condition", "Temp1[Deg]"]
    return df

def toyo_min_cap(raw_file_path, mincapacity, inirate):
    if mincapacity == 0:
        if "mAh" in raw_file_path:
            mincap = name_capacity(raw_file_path)
        else:
            inicapraw = toyo_read_csv(raw_file_path, 1)
            if inicapraw is not None and not inicapraw.empty and "Current[mA]" in inicapraw.columns:
                mincap = int(round(inicapraw["Current[mA]"].max() / inirate))
            else:
                mincap = 2000 # Default fallback
    else:
        mincap = mincapacity
    return mincap

def toyo_dchg_Profile_data(raw_file_path, inicycle, mincapacity, cutoff, inirate, smoothdegree):
    df = pd.DataFrame()
    mincapacity = toyo_min_cap(raw_file_path, mincapacity, inirate)
    if not os.path.isfile(os.path.join(raw_file_path, f"{inicycle:06d}")):
        return [mincapacity, df]
        
    tempdata = toyo_Profile_import(raw_file_path, inicycle)
    if not hasattr(tempdata, 'dataraw') or tempdata.dataraw.empty:
        return [mincapacity, df]

    df.Profile = tempdata.dataraw
    df.Profile = df.Profile[(df.Profile["Condition"] == 2)]
    df.Profile = df.Profile[df.Profile["Voltage[V]"] >= cutoff]

    if not df.Profile.empty:
        df.Profile = df.Profile.reset_index()
        df.Profile["deltime"] = df.Profile["PassTime[Sec]"].diff()
        df.Profile["delcurr"] = df.Profile["Current[mA]"].rolling(window=2).mean()
        df.Profile["delvol"] = df.Profile["Voltage[V]"].rolling(window=2).mean()
        df.Profile["delcap"] = df.Profile["deltime"] / 3600 * df.Profile["delcurr"] / mincapacity
        df.Profile["Cap[mAh]"] = df.Profile["delcap"].cumsum()
        
        if smoothdegree == 0:
            smoothdegree = int(len(df.Profile) / 30) if len(df.Profile) > 30 else 1
        
        df.Profile["delvol_smooth"] = df.Profile["Voltage[V]"].diff(periods=smoothdegree)
        df.Profile["delcap_smooth"] = df.Profile["Cap[mAh]"].diff(periods=smoothdegree)
        df.Profile["dQdV"] = df.Profile["delcap_smooth"] / df.Profile["delvol_smooth"]
        
        df.Profile["PassTime[Sec]"] = df.Profile["PassTime[Sec]"] / 60
        df.Profile["Current[mA]"] = df.Profile["Current[mA]"] / mincapacity
        df.Profile = df.Profile[["PassTime[Sec]", "Cap[mAh]", "Voltage[V]", "Current[mA]", "dQdV", "Temp1[Deg]"]]
        df.Profile.columns = ["TimeMin", "SOC", "Vol", "Crate", "dQdV", "Temp"]
    
    return [mincapacity, df]

def pne_min_cap(raw_file_path, mincapacity, ini_crate):
    if mincapacity == 0:
        if "mAh" in raw_file_path:
            return name_capacity(raw_file_path)
        restore_path = os.path.join(raw_file_path, 'Restore')
        if os.path.isdir(restore_path):
            try:
                first_file = os.path.join(restore_path, 'SaveData0001.csv')
                if os.path.exists(first_file) and os.stat(first_file).st_size != 0:
                    inicapraw = pd.read_csv(first_file, sep=",", header=None, on_bad_lines='skip')
                    if len(inicapraw) > 2:
                        return int(round(abs(inicapraw.iloc[2, 9] / 1000)) / ini_crate)
            except Exception:
                return 2000 # Default fallback
    return mincapacity

def pne_data(raw_file_path, inicycle):
    df = pd.DataFrame()
    restore_path = os.path.join(raw_file_path, 'Restore')
    if not os.path.isdir(restore_path):
        return df
    
    # Simplified file search logic
    all_files = sorted([f for f in os.listdir(restore_path) if f.startswith('SaveData') and f.endswith('.csv')])
    
    # This is a simplification. The original code has complex logic to find the exact file.
    # We will read a few files around the cycle number as a heuristic.
    # For this example, we just read all SaveData files and filter by cycle.
    all_data = []
    for f in all_files:
        try:
            chunk = pd.read_csv(os.path.join(restore_path, f), header=None, on_bad_lines='skip')
            all_data.append(chunk)
        except Exception:
            continue
    if all_data:
        df.Profileraw = pd.concat(all_data, ignore_index=True)
    return df

def pne_dchg_Profile_data(raw_file_path, inicycle, mincapacity, cutoff, inirate, smoothdegree):
    df = pd.DataFrame()
    mincapacity = pne_min_cap(raw_file_path, mincapacity, inirate)
    if mincapacity is None: mincapacity = 2000

    pnetempdata = pne_data(raw_file_path, inicycle)
    if not hasattr(pnetempdata, 'Profileraw'):
        return [mincapacity, df]

    Profileraw = pnetempdata.Profileraw
    Profileraw = Profileraw.loc[(Profileraw[27] == inicycle) & (Profileraw[2].isin([9, 2]))]
    if Profileraw.empty:
        return [mincapacity, df]

    Profileraw = Profileraw[[17, 8, 9, 11, 15, 21, 7]]
    Profileraw.columns = ["PassTime[Sec]", "Voltage[V]", "Current[mA]", "Dchgcap", "Dchgwh", "Temp1[Deg]", "step"]
    
    # Unit conversion
    Profileraw["PassTime[Sec]"] /= (100 * 60)
    Profileraw["Voltage[V]"] /= 1000000
    Profileraw["Current[mA]"] = Profileraw["Current[mA]"] / mincapacity / 1000 * (-1)
    Profileraw["Dchgcap"] /= (mincapacity * 1000)
    Profileraw["Temp1[Deg]"] /= 1000

    df.Profile = Profileraw
    df.Profile = df.Profile.reset_index()
    df.Profile = df.Profile[df.Profile["Voltage[V]"] >= cutoff]

    if smoothdegree == 0:
        smoothdegree = int(len(df.Profile) / 30) if len(df.Profile) > 30 else 1

    df.Profile["delvol"] = df.Profile["Voltage[V]"].diff(periods=smoothdegree)
    df.Profile["delcap"] = df.Profile["Dchgcap"].diff(periods=smoothdegree)
    df.Profile["dQdV"] = df.Profile["delcap"] / df.Profile["delvol"]
    
    df.Profile = df.Profile[["PassTime[Sec]", "Dchgcap", "Voltage[V]", "Current[mA]", "dQdV", "Temp1[Deg]"]]
    df.Profile.columns = ["TimeMin", "SOC", "Vol", "Crate", "dQdV", "Temp"]

    return [mincapacity, df]

### 3. Data Processing and Plotting

In [ ]:
print(f"Analyzing folder: {RAW_DATA_PATH}")
print(f"Selected cycle: {CYCLE_NUMBER}")

# Determine the cycler type
cycler_type = check_cycler(RAW_DATA_PATH)
print(f"Detected cycler type: {cycler_type}")

capacity = RATED_CAPACITY_MAH
processed_data = pd.DataFrame()

if cycler_type == 'Toyo':
    # For Toyo, the data is often in a subfolder named by channel, which is the current folder.
    # The parent folder contains the CMT file and other metadata.
    path_for_capacity_check = os.path.dirname(RAW_DATA_PATH)
    capacity, data_df = toyo_dchg_Profile_data(RAW_DATA_PATH, CYCLE_NUMBER, capacity, VOLTAGE_CUTOFF, INITIAL_C_RATE, SMOOTHING_DEGREE)
    if hasattr(data_df, 'Profile'):
        processed_data = data_df.Profile

elif cycler_type == 'PNE':
    capacity, data_df = pne_dchg_Profile_data(RAW_DATA_PATH, CYCLE_NUMBER, capacity, VOLTAGE_CUTOFF, INITIAL_C_RATE, SMOOTHING_DEGREE)
    if hasattr(data_df, 'Profile'):
        processed_data = data_df.Profile
else:
    print("Could not determine cycler type or read data.")

print(f"Used capacity for calculations: {capacity:.2f} mAh")

# Plotting
if not processed_data.empty:
    print("Data processed successfully. Generating plots...")
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    fig.suptitle(f'Discharge Profile - Cycle {CYCLE_NUMBER}', fontsize=16)

    # Voltage and Current vs. SOC
    ax1_twin = ax1.twinx()
    ax1.plot(processed_data['SOC'], processed_data['Vol'], label='Voltage (V)', color='b')
    ax1_twin.plot(processed_data['SOC'], processed_data['Crate'], label='C-rate', color='r', linestyle='--')
    ax1.set_ylabel('Voltage (V)', color='b')
    ax1_twin.set_ylabel('C-rate', color='r')
    ax1.grid(True)
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')

    # dQ/dV vs. Voltage
    ax2.plot(processed_data['Vol'], processed_data['dQdV'], label='dQ/dV', color='g')
    ax2.set_xlabel('Voltage (V)')
    ax2.set_ylabel('dQ/dV')
    ax2.grid(True)
    ax2.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    
    # Display head of the data
    print("\nProcessed Data Head:")
    print(processed_data.head())
else:
    print("\nNo data was processed. Please check the configuration and file paths.")
